In [55]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
import matplotlib.pyplot as plt
from functions import *

In [56]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre.
friction = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\new_friction.tif"
# Startpoints
supply_points = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\LPG_start.shp"
gdf_supply_points = gpd.read_file(supply_points)
#AOI
mask = r"C:\Proposals\CCA\gadm36_NPL_shp\gadm36_NPL_0.shp"
#distance raster
distance = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\distance.tif"
#new friction based on start points
new_friction = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\new_friction_layer.tif"

In [57]:
with rasterio.open(friction) as src:
    fric = src.read(1)
    nd = src.nodata
    fric = fric*1000/60

out_meta = src.meta    
fric[fric==nd] = float('inf')

In [58]:
mcp = MCP_Geometric(fric, fully_connected=True)

In [59]:
#Finding start points
row, col = friction_start_points(friction, supply_points, new_friction)

In [60]:
#Adding statvalues
pointlist = []
i = 0
for test in row:
    positiions = [row[i],col[i]]
    pointlist.append(positiions)
    i = i+1

In [61]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [63]:
new_dataset = rasterio.open(new_friction, 'w', driver='GTiff',
                            height = src.shape[0], width = src.shape[1],
                            count=1, dtype="float64",
                            crs=src.crs, transform = src.transform)

new_dataset.write(cumulative_costs, 1)
new_dataset.close()